In [1]:
import sys
sys.path.insert(0,'C:/Users/owner/Documents/IE514/Project/gym-warehouse')

import gym
import gym_warehouse
import numpy as np
import copy

env = gym.make('warehouse-v0')
# env.render()

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html
SHAPE OF CLASS MAP:  (5, 10)


In [3]:
# CNNDQN
import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# torch.manual_seed(1356)

# Device configuration
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# torch.cuda.manual_seed(1356)
# torch.backends.cudnn.deterministic = True

# out_dir = '/u/training/tra442/scratch/'
out_dir = 'outdir/'

# from wrappers import make_atari, wrap_deepmind, wrap_pytorch
# env    = make_atari(env_id)
# env    = wrap_deepmind(env)
# env    = wrap_pytorch(env)


import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import matplotlib.pyplot as plt

USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

from collections import deque

class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)

        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done

    def __len__(self):
        return len(self.buffer)


def update_target(current_model, target_model):
    target_model.load_state_dict(current_model.state_dict())

estimated_next_q_state_values=[]
estimated_next_q_value=[]

def compute_td_loss(batch_size):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)

    state      = Variable(torch.FloatTensor(np.float32(state)))
    next_state = Variable(torch.FloatTensor(np.float32(next_state)))
    action     = Variable(torch.LongTensor(action))
    reward     = Variable(torch.FloatTensor(reward))
    done       = Variable(torch.FloatTensor(done))

    q_values      = current_model(state)
    next_q_values = current_model(next_state)
    next_q_state_values = target_model(next_state)

    estimated_next_q_state_values.append(next_q_state_values)


    q_value       = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_value = next_q_state_values.gather(1, torch.max(next_q_values, 1)[1].unsqueeze(1)).squeeze(1)

    estimated_next_q_value.append(next_q_value)

    expected_q_value = reward + gamma * next_q_value * (1 - done)

    loss = (q_value - Variable(expected_q_value.data)).pow(2).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss

class CnnDQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(CnnDQN, self).__init__()

        self.input_shape = input_shape
        self.num_actions = num_actions

        self.features = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def feature_size(self):
        return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)

    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            print(q_value)
            action  = q_value.max(1)[1].data[0]
        else:
            action1 = random.randrange(5)
#             action2 = 5+random.randrange(5)
        return action1
    
    
class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_actions)
        )
        
    def forward(self, x):
        return self.layers(x)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(state).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            print(q_value)
            q_1 = q_value[0][:-5]
            q_2 = q_value[0][5:]
            print("WHEN IN DOUBT: ",q_1)
#             b[0][:-5].max(0)[0].item()
            action_1  = q_1.max(0)[1].data.item()
            action_2  = q_2.max(0)[1].data.item()
        else:
            action_1 = random.randrange(5)
            action_2 = random.randrange(5)
        return action_1,action_2

In [4]:
# current_model = CnnDQN(env.observation_space.shape, env.action_space.n)
# target_model  = CnnDQN(env.observation_space.shape, env.action_space.n)

current_model = DQN(env.observation_space.shape[0]*env.observation_space.shape[1], 10)
target_model  = DQN(env.observation_space.shape[0]*env.observation_space.shape[1], 10)


if USE_CUDA:
    current_model = current_model.cuda()
    target_model  = target_model.cuda()

optimizer = optim.Adam(current_model.parameters(), lr=0.00001)

replay_initial = 10000
replay_buffer = ReplayBuffer(10000)

update_target(current_model, target_model)

epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 100000

epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

num_frames = 100000
batch_size = 32
gamma      = 0.99

losses = []
all_rewards = []
episode_reward = 0

state = env.reset()
state = state.flatten()
# env.render()
for frame_idx in range(1, num_frames + 1):
    epsilon = epsilon_by_frame(frame_idx)
    action_1,action_2 = current_model.act(state, epsilon)

    next_state, reward, done, _ = env.step([action_1,action_2])
    next_state=next_state.flatten()
    replay_buffer.push(state, [action_1,action_2], reward, next_state, done)
#     env.render()

    state = next_state
    episode_reward += reward[0]+reward[1]

    if (frame_idx%100000==0):
        # SAVING LOSS AND REWARD
        np.save(out_dir + 'losses_ddqn_seed0.npy',np.array(losses))
        np.save(out_dir + 'rewards_ddqn_seed0.npy',np.array(all_rewards))

#         SAVING CHECKPOINTS
        torch.save(current_model.state_dict(),out_dir + 'cur_ckp_ddqn/current{}_seed0_DDQN.ckpt'.format(frame_idx))
        torch.save(target_model.state_dict(),out_dir + 'tar_ckp_ddqn/target{}_seed0_DDQN.ckpt'.format(frame_idx))

#         SAVING MODELS
        torch.save(current_model,out_dir + 'cur_mod_ddqn/current{}_seed0_DDQN.model'.format(frame_idx))
        torch.save(target_model,out_dir + 'tar_mod_ddqn/target{}_seed0_DDQN.model'.format(frame_idx))

#         SAVING ESTIMATED Q VALUES
        np.save(out_dir + 'next_q_state_values_ddqn_seed0.npy',np.array(estimated_next_q_state_values))
        np.save(out_dir + 'next_q_value_ddqn_seed0.npy',np.array(estimated_next_q_value))

    if done:
        state = env.reset()
        all_rewards.append(episode_reward)
        episode_reward = 0

    if frame_idx % 10000==0:
        print('Frame: ',frame_idx)

    if (len(replay_buffer) > replay_initial) and (frame_idx%4==0):
        loss = compute_td_loss(batch_size)
        losses.append(loss.item())

#     if frame_idx % 100000 == 0:
#         plot(frame_idx, all_rewards, losses)

    if frame_idx % 10000 == 0:
        update_target(current_model, target_model)

ACTION IS:  [4, 0]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 0]), array([3, 0])]
Number of Orders Fulfilled:  0
Reward:  [-0.002, -10]
Self.is_loaded :  [False, False]
ACTION IS:  [4, 0]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 0]), array([3, 0])]
Number of Orders Fulfilled:  0
Reward:  [-10, -10]
Self.is_loaded :  [False, False]
ACTION IS:  [2, 2]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 0]), array([3, 0])]
Number of Orders Fulfilled:  0
Reward:  [-10, -10]
Self.is_loaded :  [False, False]
ACTION IS:  [1, 0]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 1]), array([3, 0])]
Number of Orders Fulfilled:  0
Reward:  [-0.002, -10]
Self.is_loaded :  [False, False]
ACTION IS:  [0, 4]
Old Load:  [False, False]
New L

Robot:  [array([1, 4]), array([1, 6])]
Number of Orders Fulfilled:  1
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, True]
ACTION IS:  [4, 0]
Old Load:  [False, True]
New Load:  [False, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 4]), array([1, 6])]
Number of Orders Fulfilled:  1
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, True]
ACTION IS:  [2, 1]
Old Load:  [False, True]
New Load:  [False, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 3]), array([1, 7])]
Number of Orders Fulfilled:  1
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, True]
ACTION IS:  [1, 0]
Old Load:  [False, True]
New Load:  [False, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 4]), array([1, 7])]
Number of Orders Fulfilled:  1
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, True]
ACTION IS:  [4, 2]
Old Load:  [False, True]
New Load:  [False, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 4]), array([1, 6])]
Numbe

Self.is_loaded :  [True, True]
ACTION IS:  [2, 0]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 4]), array([2, 8])]
Number of Orders Fulfilled:  1
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [4, 0]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 4]), array([2, 8])]
Number of Orders Fulfilled:  1
Reward:  [-5, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [2, 2]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 3]), array([2, 7])]
Number of Orders Fulfilled:  1
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [4, 2]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 3]), array([2, 6])]
Number of Orders Fulfilled:  1
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [3, 1]
Old Load: 

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


ACTION IS:  [1, 4]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 1]), array([2, 0])]
Number of Orders Fulfilled:  6
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, False]
ACTION IS:  [0, 2]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 1]), array([2, 0])]
Number of Orders Fulfilled:  6
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, False]
ACTION IS:  [2, 2]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 0]), array([2, 0])]
Number of Orders Fulfilled:  6
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, False]
ACTION IS:  [4, 0]
Old Load:  [False, False]
New Load:  [False, False]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 0]), array([2, 0])]
Number of Orders Fulfilled:  6
Reward:  [-0.002, -0.002]
Self.is_loaded :  [False, False]
ACTION IS:  [1, 3]
Old Load:  [F

...

New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 3]), array([3, 5])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [3, 4]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 3]), array([4, 5])]
Number of Orders Fulfilled:  114
Reward:  [-5, -0.002]
Self.is_loaded :  [True, True]
tensor([[ 0.0419,  0.1182,  0.0416,  0.0029,  0.0577, -0.0813, -0.0009,  0.0147,
          0.0746,  0.0305]], grad_fn=<AddmmBackward>)
WHEN IN DOUBT:  tensor([0.0419, 0.1182, 0.0416, 0.0029, 0.0577], grad_fn=<SliceBackward>)
ACTION IS:  [1, 3]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 4]), array([3, 5])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [1, 0]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]

Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([1, 7]), array([0, 5])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [4, 1]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 7]), array([0, 6])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
tensor([[ 0.0351,  0.1700,  0.0542,  0.0201,  0.1559, -0.0805, -0.0252,  0.0133,
          0.0454,  0.0889]], grad_fn=<AddmmBackward>)
WHEN IN DOUBT:  tensor([0.0351, 0.1700, 0.0542, 0.0201, 0.1559], grad_fn=<SliceBackward>)
ACTION IS:  [1, 4]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 8]), array([1, 6])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [4, 1]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 

Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 9]), array([2, 5])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [2, 1]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([3, 8]), array([2, 6])]
Number of Orders Fulfilled:  114
Reward:  [-5, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [3, 1]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 8]), array([2, 7])]
Number of Orders Fulfilled:  114
Reward:  [-5, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [2, 0]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([2, 7]), array([2, 7])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [3, 3]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array

WHEN IN DOUBT:  tensor([ 0.0033,  0.1271, -0.0270,  0.0174,  0.1430], grad_fn=<SliceBackward>)
ACTION IS:  [4, 3]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 8]), array([0, 5])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -5]
Self.is_loaded :  [True, True]
tensor([[-0.0074,  0.1210,  0.0001,  0.0499,  0.1051, -0.0779, -0.0100, -0.0097,
          0.0290,  0.0155]], grad_fn=<AddmmBackward>)
WHEN IN DOUBT:  tensor([-0.0074,  0.1210,  0.0001,  0.0499,  0.1051], grad_fn=<SliceBackward>)
ACTION IS:  [1, 3]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 9]), array([0, 5])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.002]
Self.is_loaded :  [True, True]
ACTION IS:  [1, 2]
Old Load:  [True, True]
New Load:  [True, True]
Entrance:  [array([1, 0]), array([3, 0])]
Robot:  [array([4, 9]), array([0, 4])]
Number of Orders Fulfilled:  114
Reward:  [-0.002, -0.0

KeyboardInterrupt: 

In [ ]:
env.action_space

In [ ]:
b=torch.tensor([[-0.0814,  0.0060, -0.0667,  0.0217,  0.0114, -0.1569, -0.0728,  0.1463,
          0.0184, -0.1267]])

In [ ]:
b[0][-5:]

In [ ]:
b[0][:-5].max(0)[0].item()